In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

import os
import subprocess
import re

root = None
repository = None
label = None
run_button = None
scrollable_frame = None
# Refactor out of UI
energibridge_path = os.path.join(os.getcwd(), "energibridge", "energibridge.exe")

def update_label():
    label.config(text=f"Running energibridge...", foreground="#4CAF50")  # Green text
    
# Refactor out of UI
def find_gradle_build(folder):
    for root, _, files in os.walk(folder):
        if "build.gradle" in files or "build.gradle.kts" in files:
            return os.path.join(root, "build.gradle") if "build.gradle" in files else os.path.join(root, "build.gradle.kts")
    return None
    
def browse_folder():
    global repository
    folder_selected = filedialog.askdirectory()
    if folder_selected:
        gradle_file = find_gradle_build(folder_selected)
        if gradle_file:
            label.config(text=f"Found: {gradle_file}")
            repository = folder_selected
            run_button.config(state='normal')
            updateTaskList()  
        else:
            label.config(text="gradle.build file not found")
            repository = None
            run_button.config(state='disabled')

            
# Refactor out of UI        
def getTasks():
    command = f"gradle build --rerun-tasks --dry-run"
    result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd=repository)
    print("hi")
    regex = re.compile(r"^:(\S+) SKIPPED")
    tasks = []

    for line in result.stdout.splitlines():
        match = regex.match(line)
        if match:
            task = match.group(1) 
            print(f"Found task: {task}")
            tasks.append(task)
    return tasks

# Refactor out of UI
def runTasks(tasks):
    for task in tasks:
        print(f"Running task: {task}")
        #command = f"{energibridge_path} --summary cmd /c \"gradle {task}\" --iterations {iterations_entry.get()} --sleep {sleep_entry.get()} --interval {interval_entry.get()}"
        command = f"{energibridge_path} --summary cmd /c \"gradle {task}\""
        result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd=repository)
        print(result)
            
            
def runBridge():
    tasks = getTasks()
    runTasks(tasks)
    update_label()
    
def updateTaskList():
    global scrollable_frame
    for widget in scrollable_frame.winfo_children():
        widget.destroy()
    vars = []
    task_list = getTasks()
    for task in task_list:
        var = tk.BooleanVar()
        vars.append(var)
        chk = ttk.Checkbutton(scrollable_frame, text=task, variable=var)
        chk.pack(anchor="w", pady=2, fill="both")
    root.update_idletasks()
    
    

def main():
    global root, label, repository, run_button, scrollable_frame
    # Main window
    root = tk.Tk()
    root.title("GradleBridge")
    #root.geometry("800x500")
    root.configure(bg="#f8f9fa")

    #Style
    style = ttk.Style()
    style.theme_use("clam")  # Alternative themes: "alt", "default", "classic"
    style.configure("TFrame", background="#f8f9fa")
    style.configure("TLabel", font=("Arial", 14, "bold"), background="#f8f9fa", foreground="#333")
    style.configure("TEntry", font=("Arial", 12), padding=5)
    style.configure("run.TButton", font=("Arial", 12, "bold"), background="#4CAF50", foreground="white", padding=10)
    #style.map("run.TButton", background=[("active", "#45a049")]) 
    style.map("run.TButton", background=[("disabled", "#e0e0e0"), ("active", "#45a049")], foreground=[("disabled", "gray")])

    style.configure("browse.TButton", font=("Arial", 12, "bold"), background="#2196F3", foreground="white", padding=10)
    style.map("browse.TButton", background=[("active", "#1976D2")])  # Darker blue when hovered
    

    main_frame = ttk.Frame(root, style="TFrame", padding=20)
    main_frame.pack(expand=True, fill="both", anchor="center")

    #Entries (fields)    
    ttk.Label(main_frame, text="Iterations:", style="TLabel").pack(pady=5)
    iterations_entry = ttk.Entry(main_frame, style="TEntry")
    iterations_entry.pack(pady=5)

    ttk.Label(main_frame, text="Sleep (seconds):", style="TLabel").pack(pady=5)
    sleep_entry = ttk.Entry(main_frame, style="TEntry")
    sleep_entry.pack(pady=5)

    ttk.Label(main_frame, text="Interval (seconds):", style="TLabel").pack(pady=5)
    interval_entry = ttk.Entry(main_frame, style="TEntry")
    interval_entry.pack(pady=5)

    #Buttons
    browse_button = ttk.Button(main_frame, text="Select Folder", command=browse_folder, style="browse.TButton")
    browse_button.pack(side=tk.LEFT, expand=True)
    
    run_button = ttk.Button(main_frame, text="Run", command=runBridge, style="run.TButton", state='disabled')
    run_button.pack(side=tk.LEFT, expand=True)

    
    
    label = ttk.Label(main_frame, text="", style="TLabel")
    label.pack(pady=20)
    
    container = ttk.Frame(root)
    canvas = tk.Canvas(container, bg="white")
    
    def on_mouse_wheel(event):
        canvas.yview_scroll(-1 * (event.delta // 120), "units")
        
    canvas.bind_all("<MouseWheel>", on_mouse_wheel) 
    canvas.bind_all("<Button-4>", on_mouse_wheel)  
    canvas.bind_all("<Button-5>", on_mouse_wheel) 
    
    scrollbar = ttk.Scrollbar(container, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    container.pack(fill="both", expand=True)
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Create checkboxes inside the scrollable frame
    vars = []
    task_list = [f"Task {i}" for i in range(50)]
    for task in task_list:
        var = tk.BooleanVar()
        vars.append(var)
        chk = ttk.Checkbutton(scrollable_frame, text=task, variable=var)
        chk.pack(anchor="w", pady=2, fill="both")

    root.mainloop()

if __name__ == "__main__":
    main()


hi
Found task: compileJava
Found task: processResources
Found task: classes
Found task: jar
Found task: javadoc
Found task: javadocJar
Found task: sourcesJar
Found task: assemble
Found task: compileTestJava
Found task: processTestResources
Found task: testClasses
Found task: test
Found task: check
Found task: build
hi
hi
Found task: compileJava
Found task: processResources
Found task: classes
Found task: jar
Found task: javadoc
Found task: javadocJar
Found task: sourcesJar
Found task: assemble
Found task: compileTestJava
Found task: processTestResources
Found task: testClasses
Found task: test
Found task: check
Found task: build
hi
Found task: compileJava
Found task: processResources
Found task: classes
Found task: jar
Found task: startScripts
Found task: distTar
Found task: distZip
Found task: assemble
Found task: compileTestJava
Found task: processTestResources
Found task: testClasses
Found task: test
Found task: check
Found task: build
hi
Found task: compileJava
Found task: processR